In [23]:
from alphaverse.expr import And, Gt, Lt, Constant, Factor, VectorMult, TopK
from alphaverse.engine import Engine
from provider import PriceProvider, SMAProvider, NewPriceProvider, SmaMomentumProvider, RSProvider

In [17]:
providers = {}
providers["close"] = PriceProvider("close")
providers["sma50"] = SMAProvider(50)
providers["sma150"] = SMAProvider(150)
providers["sma200"] = SMAProvider(200)
providers["high52"] = NewPriceProvider(field="high", window=52)
providers["low52"] = NewPriceProvider(field="low", window=52)
providers["sma_momentum"] = SmaMomentumProvider(200, 20)
providers["rs"] = RSProvider()

In [18]:
engine = Engine(providers)

In [19]:
# trend template conditions
# 1. 현 주가가 150일(30주), 이동평균선이 200일(40주) 이동평균선 위에 있다.
# 2. 150일 이동평균선이 200일 이동평균선 위에 있다.
# 3. 200일 이동평균선이 적어도 1개월 동안(최소 4~5개월이 바람직함) 상승 추세에 있다.
# 4. 50일(10주) 이동평균선이 150일, 200일 이동평균선보다 위에 있다.
# 5. 현 주가가  50일 이동평균선 위에 있다.
# 6. 현 주가가 52주 신저가보다 30% 위에 있다.
# 7. 현 주가가 최소한 52주 신고가의 25% 안에 있다.(신고가에 가까울 수록 좋다.)
# 8. 중요 지수 대비 개별종목의 상대적 상승 및 하락여부를 계산하여 1~99까지 랭킹을 매긴다. 숫자가 높을 수록 지수대비 상대 강도가 강한 종목이라고 볼 수 있다.

In [32]:
fst = And(Gt(Factor('close'), Factor('sma150')), Gt(Factor('close'), Factor('sma200')))
secd = Gt(Factor('sma150'), Factor('sma200'))
thrd = Gt(Factor("sma_momentum"), Constant(0))
forth = And(Gt(Factor('sma50'), Factor('sma150')), Gt(Factor('sma50'), Factor('sma200')))
fifth = Gt(Factor('close'), Factor('sma50'))
sixth = Gt(Factor('close'), VectorMult(Factor('low52'), Constant(1.3)))
seventh = And(Gt(Factor('close'), VectorMult(Factor('high52'), Constant(0.75))), Lt(Factor('close'), VectorMult(Factor('high52'), Constant(1.25))))
eighth = TopK(Factor('rs'), Constant(99))

In [33]:
expr = And(fst, secd, thrd, forth, fifth, sixth, seventh, eighth)

In [34]:
await engine.screen(expr)

code,close,sma150,sma200,sma_momentum,sma50,low52,high52,rs
str,f64,f64,f64,f64,f64,f64,f64,f64
"""700009""",20565.0,13990.166667,13832.8,125.0,16136.1,11610.0,20565.0,59.762707
"""179900""",37800.0,32690.666667,30364.4,950.0,32950.0,17320.0,42550.0,57.759825
"""530063""",20795.0,14091.866667,13950.45,210.0,16254.8,11610.0,20795.0,59.056908
"""500040""",26780.0,17673.433333,17433.5,425.0,20690.4,14525.0,33300.0,61.105074
"""078350""",28250.0,16275.0,15493.1,200.0,20532.2,11200.0,30800.0,69.971606
"""232140""",16380.0,6181.633333,5633.875,90.0,9150.0,3120.0,16930.0,128.134279
"""252990""",7890.0,6370.5,6200.275,630.0,7385.6,4585.0,9280.0,106.590223
"""106190""",13030.0,9587.266667,9526.67,140.0,10194.6,7880.0,13030.0,67.744774
"""223220""",650.0,494.84,491.925,40.0,546.5,308.0,698.0,127.546715
